# Passive AllenNLP SRL BiLSTM

In [1]:
from allennlp_models.pretrained import load_predictor

/home/elena/anaconda3/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
import csv

In [5]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [6]:
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Someone was cuddled by Peter.")
output

{'verbs': [{'verb': 'was',
   'description': '[ARG1: Someone] [V: was] [ARG2: cuddled by Peter] .',
   'tags': ['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']},
  {'verb': 'cuddled',
   'description': '[ARG0: Someone] was [V: cuddled] [ARG0: by Peter] .',
   'tags': ['B-ARG0', 'O', 'B-V', 'B-ARG0', 'I-ARG0', 'O']}],
 'words': ['Someone', 'was', 'cuddled', 'by', 'Peter', '.']}

In [7]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [8]:
d = ["Someone was cuddled by Peter."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'was',
     'description': '[ARG1: Someone] [V: was] [ARG2: cuddled by Peter] .',
     'tags': ['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']},
    {'verb': 'cuddled',
     'description': '[ARG0: Someone] was [V: cuddled] [ARG0: by Peter] .',
     'tags': ['B-ARG0', 'O', 'B-V', 'B-ARG0', 'I-ARG0', 'O']}],
   'words': ['Someone', 'was', 'cuddled', 'by', 'Peter', '.']}],
 array([1.]))

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description']

In [10]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [11]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']
        
    return predicate_structure

In [12]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [13]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} was cuddled by Peter." , meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/passive_arg1_cuddled_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/passive_arg1_cuddled_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 500 examples
Test cases:      500
Fails (rate):    384 (76.8%)

Example fails:
[ARG0: Carol] was [V: cuddled] [ARG0: by Peter] .
----
[ARG0: Diana] was [V: cuddled] [ARG0: by Peter] .
----
[ARG2: Steven] was [V: cuddled] [ARG0: by Peter] .
----
['Alice', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Samuel', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Joe', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Stephanie', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Alison', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Peter', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Pamela', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Harry', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Bruce', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Stephanie', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Marie', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Martha', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['David', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Walt

In [14]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} was hugged by Peter." , meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/passive_arg1_hugged_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/passive_arg1_hugged_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 500 examples
Test cases:      500
Fails (rate):    4 (0.8%)

Example fails:
Kathy was [V: hugged] [ARG0: by Peter] .
----
Kathy was [V: hugged] [ARG0: by Peter] .
----
Kathy was [V: hugged] [ARG0: by Peter] .
----
['Donald', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Paul', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Annie', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Richard', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Thomas', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Alan', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Eric', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Steve', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Anthony', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Charlotte', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Leslie', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Peter', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Donald', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Virginia', 'was', 'hugged', 'by', 'Peter', '.'] [

In [15]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} was adored by Peter." , meta=True,nsamples=500, remove_duplicates=True)


In [16]:
with open('/datasets/passive_arg1_adored_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/passive_arg1_adored_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 500 examples
Test cases:      500
Fails (rate):    5 (1.0%)

Example fails:
[ARGM-TMP: Steve] was [V: adored] [ARG0: by Peter] .
----
[ARGM-TMP: Nicole] was [V: adored] [ARG0: by Peter] .
----
[ARGM-TMP: Nicole] was [V: adored] [ARG0: by Peter] .
----
['Lawrence', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Michael', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Joe', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['James', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Diana', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Emma', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Charlie', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Roger', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Patrick', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Tony', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Donna', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Stephen', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Diane', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Pamela', 'was

In [17]:
verbs = ['hugged', 'kissed', 'loved', 'appreciated', 'adored', 'cuddled']

In [18]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} was {verb} by Peter.", verb = verbs, meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/passive_arg1_verb_variation_bilstm.txt', 'w') as f:
    print(t.data, file = f)


# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/passive_arg1_verb_variation_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 500 examples
Test cases:      500
Fails (rate):    60 (12.0%)

Example fails:
[ARG0: Carl] was [V: cuddled] [ARG0: by Peter] .
----
[ARGM-MNR: Kathy] was [V: cuddled] [ARG0: by Peter] .
----
[ARG0: Diana] was [V: cuddled] [ARG0: by Peter] .
----
['Virginia', 'was', 'loved', 'by', 'Peter', '.'] [ True]
['Mike', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Colin', 'was', 'cuddled', 'by', 'Peter', '.'] [False]
['Sue', 'was', 'appreciated', 'by', 'Peter', '.'] [ True]
['Lisa', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Gary', 'was', 'loved', 'by', 'Peter', '.'] [ True]
['Charles', 'was', 'loved', 'by', 'Peter', '.'] [ True]
['Arthur', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Bobby', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Dan', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Jane', 'was', 'loved', 'by', 'Peter', '.'] [ True]
['Ron', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Alan', 'was', 'loved', 'by', 'Peter', '.'] [ True]
['Frank', 'was', 'hugged', 'by'

# End of Notebook